# Notebook to loop over events generated in Pythia's c++ version


In [1]:
import numpy as np
import sys
import dimod
import matplotlib.pyplot as plt
import os
import re
import neal

from dwave.system import EmbeddingComposite, DWaveSampler, ReverseAdvanceComposite
from dwave_qbsolv import QBSolv
from pyqubo import Spin, Array
from embedding_generator import generate_embedding

In [2]:
# Define variable here

# Observed big difference in obtained minimum when increasing the number of reads from 100 to 1000
num_reads = 10000
num_events = 1000
path = '/Users/1zd/quantumjets/pythiaEvents/'

theta = np.pi/2.

sampler = EmbeddingComposite(DWaveSampler(solver='Advantage_system1.1'))
sa_sampler = neal.SimulatedAnnealingSampler()

In [12]:

with open('/Users/1zd/quantumjets/pythiaEvents/DWaveResults_tune_10_4.dat', 'w') as fout:
    for ev in range(879, num_events):
        print(ev)

        fname = path + 'Event_' + str(ev) + '.dat'
        f = open(fname)

        pat = re.compile(r"\s+")

        g = list(zip(*[ pat.sub(" ",x.strip()).split() for x in f]))

        px = np.array([ float(x) for x in g[0]])
        py = np.array([ float(x) for x in g[1]])
        pz = np.array([ float(x) for x in g[2]])
        e = np.array([ float(x) for x in g[3]])

        # Generate qubo

        n_part = len(px)
        #print(n_part)
        s = Array.create('s', shape=n_part, vartype='BINARY')

        # Create an array of zeroes for qubo matrix coefficients
        coeff = [[0] * n_part for _ in range(n_part)] 

        for i in range(0,n_part):
            for j in range(0,n_part):
                coeff[i][j] = (px[i]*px[j] +py[i]*py[j] + pz[i]*pz[j] - e[i]*e[j] * np.cos(theta))/(1 - np.cos(theta))

        # Construct Hamiltonian
        #H = sum([ ((1.0/(np.sqrt(px[i]*px[i] +py[i]*py[i] + pz[i]*pz[i])*np.sqrt(px[j]*px[j] +py[j]*py[j] + pz[j]*pz[j]))) - 1.0)*coeff[i][j]*s[i]*s[j] for i in range(0,n_part) for j in range(0,n_part)])
        H = sum([ -1.0*coeff[i][j]*s[i]*s[j] for i in range(0,n_part) for j in range(0,n_part)])

        # Compile model using pyqubo

        model = H.compile()
        qubo, offset = model.to_qubo()

        # Submit problem to D-Wave
        #response = QBSolv().sample_qubo(qubo)#
        ch_str = qubo[max(qubo, key=qubo.get)]*0.1
        response = sampler.sample_qubo(qubo,num_reads=10, chain_strength=ch_str, annealing_time=100)
        #sampler_reverse = ReverseAdvanceComposite(sampler)
        #schedule = [[[0.0, 1.0], [t, 0.5], [20, 1.0]] for t in (5, 10, 15)]
        #sampleset = sampler_reverse.sample_qubo(qubo, anneal_schedules=schedule, num_reads=100,reinitialize_state=True) 


        #fnameout = '/Users/1zd/quantumjets/pythiaEvents/Event_%s_qsolutions_v2.dat' %ev
        #with open(fnameout, 'w') as f:
        #    for p in range(n_part):
        #        key = 's[%s]' %p
        #        f.write("%s\n" %(response.first.sample[key]))
    
    #print('Finished annealing run on Advantage 1.1')

        #print(response.first.energy)

        #response_sa = sa_sampler.sample_qubo(qubo)

    #print('Finished simmulated annealing routine')

    #print(response_sa.first.energy)

        sum_mom = sum([np.sqrt(px[i]*px[i] + py[i]*py[i]+ pz[i]*pz[i])  for i in range(0,n_part)])
    
        #print(sum_mom)
        thrust = 2.*np.sqrt(-1.*response.first.energy)/sum_mom
        print(thrust)
        fout.write("%s \n" %(thrust))

        #rev = 2.*np.sqrt(-1.*sampleset.first.energy)/sum_mom
        #print(rev)

        #thrust_sa = 2.*np.sqrt(-1.*response_sa.first.energy)/sum_mom
        #fout.write("%s  %s  %s \n" %(thrust,thrust_sa,rev))
    
    #print(response.data_vectors['sample'][0])
    # Store solutions

    #qpu_solutions = []

    #for datum in response.data(['sample','energy']):
    #    print(datum.energy)
    #    qpu_solutions.append(datum.sample)


879
0.9758157835200015
880
0.9643092826718995
881
0.9193216635293281
882
0.909658365146773
883
0.9815093898652641
884
0.9418056216670513
885
0.9877154026085957
886
0.9655808867814615
887
0.9564155143432478
888
0.9848425554811664
889
0.8178541065524306
890
0.9598596432383872
891
0.9740629886017179
892
0.8954258462870681
893
0.9884982420976204
894
0.975941615509834
895
0.8829427565557847
896
0.9715899846510833
897
0.970458084090086
898
0.7808089220828319
899
0.9482832645771059
900
0.9387383085439449
901
0.9502668060938557
902
0.8796553649810869
903
0.9488316871753673
904
0.9628730042367323
905
0.9776637704234038
906
0.826716775256079
907
0.8788763923750782
908
0.9614701067455119
909
0.8767310991591124
910
0.9846924034201318
911
0.9606871446627019
912
0.9580941781964545
913
0.9601751402235036
914
0.9700740971295608
915
0.8770685602915013
916
0.8944556622841857
917
0.9034887892357913
918
0.8787354361390506
919
0.9492594918211192
920
0.9734400462009538
921
0.6182608723628659
922
0.628465228

In [10]:
with open('/home/andrea/pythiaEvents/Event_0_qsolutions.dat', 'w') as f:
    for p in range(n_part):
        key = 's[%s]' %p
        f.write("%s\n" %(response.first.sample[key]))

In [17]:
p = 0
response.first.sample['s[7]']

0

In [ ]:
thrust = sum([ -1.0*coeff[i][j]*response.first.sample['s['+str(i)+']']*response.first.sample['s['+str(j)+']'] for i in range(0,n_part) for j in range(0,n_part)])
print(thrust)

In [24]:
sum_mom = sum([np.sqrt(px[i]*px[i] + py[i]*py[i]+ pz[i]*pz[i])  for i in range(0,n_part)])

In [25]:
print(sum_mom)

90.86611912138152


In [ ]:
-thrust/(2*sum_mom)

In [1]:
qubo

NameError: name 'qubo' is not defined